# Apply EfficientNetB0

Import Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

Data Generator

In [ ]:
train_dir = 'Split_data/train'
validation_dir = 'Split_data/val'
test_dir = 'Split_data/test'

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Important for test set to evaluate model properly
)


Found 8608 images belonging to 4 classes.
Found 1847 images belonging to 4 classes.
Found 1845 images belonging to 4 classes.


Model Definition and Compilation

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model initially

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


Model Training

In [4]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)


C:\Users\Mubasher Manzoor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7593 - loss: 0.6266

C:\Users\Mubasher Manzoor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


269/269 ━━━━━━━━━━━━━━━━━━━━ 506s 2s/step - accuracy: 0.7594 - loss: 0.6261 - val_accuracy: 0.8424 - val_loss: 0.4329
Epoch 2/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 490s 2s/step - accuracy: 0.8479 - loss: 0.3781 - val_accuracy: 0.8506 - val_loss: 0.3894
Epoch 3/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 487s 2s/step - accuracy: 0.8665 - loss: 0.3518 - val_accuracy: 0.8592 - val_loss: 0.3731
Epoch 4/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 486s 2s/step - accuracy: 0.8680 - loss: 0.3398 - val_accuracy: 0.8565 - val_loss: 0.3577
Epoch 5/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 482s 2s/step - accuracy: 0.8763 - loss: 0.3207 - val_accuracy: 0.8619 - val_loss: 0.3418
Epoch 6/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 486s 2s/step - accuracy: 0.8776 - loss: 0.3046 - val_accuracy: 0.8614 - val_loss: 0.3579
Epoch 7/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 482s 2s/step - accuracy: 0.8853 - loss: 0.2971 - val_accuracy: 0.8674 - val_loss: 0.3400
Epoch 8/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 573s 2s/step - accuracy: 0.8800 - loss: 0.2969 - val_accuracy: 0.870

Test Accuracy

In [5]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")


58/58 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.8410 - loss: 0.4790
Test Accuracy: 0.8834688067436218


Image Loading and Preprocessing

In [12]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
    img_array = preprocess_input(img_array)  # Preprocess the image according to EfficientNet standards
    return img_array


Load Model

In [ ]:
from tensorflow.keras.models import load_model

# Load the previously saved model
model = load_model('infer_model.keras')


Predict Image Class

In [16]:
def predict_image_class(img_path, model, class_indices, threshold=0.6):
    """
    Predict the class of an image. If the model's confidence is below the threshold, classify as "Unknown".
    
    Args:
        img_path (str): Path to the input image.
        model (Model): Trained model to use for prediction.
        class_indices (dict): Class indices from the training data.
        threshold (float): Confidence threshold for determining if the prediction is "Unknown".
    
    Returns:
        str: Predicted class label or "Unknown".
    """
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
    img_array = preprocess_input(img_array)  # Preprocess the image according to EfficientNet standards

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)  # Index of the highest probability
    confidence = np.max(predictions)  # Highest probability value

    # Get the class labels (only actual class names, not folder names)
    labels = {0: "Abnormal", 1: "Infection", 2: "Ischaemia", 3: "Normal"}

    # Check confidence threshold
    if confidence < threshold:
        return "Unknown"  # Confidence is below the threshold
    else:
        return labels[predicted_class_index[0]]  # Return the predicted class label


# Assuming that you have `train_generator.class_indices` and it correctly maps to the actual classes
class_indices = train_generator.class_indices

# Test image
img_path = 'Split_data/test/resized_images_of_Ischaemia/resized_image_22.jpg'  # Provide a path to an image not in your training classes
predicted_class = predict_image_class(img_path, model, class_indices, threshold=0.7)
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Predicted class: Ischaemia


Save Model

In [17]:
# Save model in .keras format
model.save("infer_model.keras")


Load Model

In [18]:
# Load the saved model
loaded_model = load_model("infer_model.keras")


C:\Users\Mubasher Manzoor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
